In [ ]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1614674878622606561, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15703311680
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13703323498338992684
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# added Google Drive directory containing scripts and data, and updated throughout the notebook

import sys

DIR = '/content/drive/My Drive/Colab Notebooks/NNDL/Project/unet/'
sys.path.append(DIR)

In [ ]:
from model import *
from data import *

## Train your Unet with membrane data
membrane data is in folder membrane/, it is a binary classification task.

The input shape of image and mask are the same :(batch_size,rows,cols,channel = 1)

### Train with data generator

In [ ]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')

trainGen = trainGenerator(2, DIR+'data/membrane/train', 'image', 'label', data_gen_args, 
                          save_to_dir=None) # save_to_dir=DIR+'data/membrane/train/aug') 

In [ ]:
model = unet() # unet(pretrained_weights=DIR+'checkpoints/unet_membrane.weights.04-0.12.hdf5')

In [ ]:
model_checkpoint = ModelCheckpoint(DIR+'checkpoints/unet_membrane.weights.{loss:.3f}-{epoch:02d}.hdf5', 
                                   monitor='loss', verbose=1, save_best_only=True, save_weights_only=True)

In [ ]:
# changed steps_per_epoch and epochs {AC}
model.fit(trainGen, steps_per_epoch=100, epochs=25, callbacks=[model_checkpoint])

Epoch 1/25
100/100 [==============================] - 11s 90ms/step - loss: 0.4760 - accuracy: 0.7860

Epoch 00001: loss improved from inf to 0.42092, saving model to /content/drive/My Drive/Colab Notebooks/NNDL/Project/unet/checkpoints/unet_membrane.weights.0.421-01.hdf5
Epoch 2/25
100/100 [==============================] - 9s 91ms/step - loss: 0.3483 - accuracy: 0.8331

Epoch 00002: loss improved from 0.42092 to 0.34370, saving model to /content/drive/My Drive/Colab Notebooks/NNDL/Project/unet/checkpoints/unet_membrane.weights.0.344-02.hdf5
Epoch 3/25
100/100 [==============================] - 9s 94ms/step - loss: 0.3259 - accuracy: 0.8515

Epoch 00003: loss improved from 0.34370 to 0.31351, saving model to /content/drive/My Drive/Colab Notebooks/NNDL/Project/unet/checkpoints/unet_membrane.weights.0.314-03.hdf5
Epoch 4/25
100/100 [==============================] - 9s 91ms/step - loss: 0.3217 - accuracy: 0.8551

Epoch 00004: loss did not improve from 0.31351
Epoch 5/25
100/100 [======

In [ ]:
# model.fit(trainGen, steps_per_epoch=20, epochs=10, callbacks=[model_checkpoint])

In [ ]:
# model.fit(trainGen, steps_per_epoch=750, epochs=5, callbacks=[model_checkpoint])

### Train with npy file

In [ ]:
# imgs_train, imgs_mask_train = trainDataNpy(DIR+"data/membrane/train/aug/", DIR+"data/membrane/train/aug/")
# model.fit(imgs_train, imgs_mask_train, batch_size=2, epochs=10, verbose=1, validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])

### test your model and save predicted results

In [ ]:
testGen = testGenerator(DIR+"data/membrane/test")
# model = unet()
# model.load_weights(DIR+"checkpoints/unet_membrane.hdf5")
results = model.predict(testGen, 30, verbose=1)
saveResult(DIR+"data/membrane/test/predict", results)

30/30 [==============================] - 19s 634ms/step


In [ ]:
model.evaluate(results)

1/1 [==============================] - 0s 266ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


[0.0, 0.0]